In [7]:
import cv2
import numpy as np

def stabilize_video(input_video, template_region, output_video): 
    cap = cv2.VideoCapture(input_video)
    template = cv2.imread(template_region, 0)
    template_height, template_width = template.shape

    # Define the output codec and create VideoWriter object 
    fourcc = cv2.VideoWriter_fourcc(*'XVID') 
    out = cv2.VideoWriter(output_video, fourcc, 30.0, (template_width, template_height))

    # Read the first frame
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale 
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform template matching
        result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result) 
        top_left = max_loc
        bottom_right = (top_left[0] + template_width, top_left[1] + template_height)

        # Calculate the motion vector
        motion_vector = (top_left[0] - template_width // 2, top_left[1] - template_height // 2)

        # Correct the frame using the motion vector
        stabilized_frame = np.roll(frame, motion_vector, axis=(0, 1))

        # Write the stabilized frame to the output video
        out.write(stabilized_frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
stabilize_video('input_video.mp4', 'template.png', 'stabilized_video.avi')
